# A simple chain for getting answers from a database

### Load the environment variables needed for the application

In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

### Connect to a database to have an access to the data

In [ ]:
from langchain_community.utilities import SQLDatabase

# Define the database URI
db_uri = "sqlite:///chinook.db"

# Create a database object
db = SQLDatabase.from_uri(db_uri)

# Test the connection by printing dialect, listing table names and querying the database
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

### Instantiate LLM model

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

### Load the tools

In [4]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

# Load the SQL tools for AI agent to use
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

### Define a State class to preserve all the required valriables accross the steps

In [5]:
# from typing_extensions import TypedDict


# class State(TypedDict):
#     question: str
#     query: str
#     result: str
#     answer: str

from langgraph.graph import MessagesState

### Pull the prebuilt prompt template for sql query generation

In [6]:
# from langchain import hub

# system_message = hub.pull("langchain-ai/sql-agent-system-prompt")

# assert len(system_message.messages) == 1
# system_message.messages[0].pretty_print()

message_template = """
    You are an agent designed to interact with a SQL database.
    Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
    Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
    You can order the results by a relevant column to return the most interesting examples in the database.
    Never query for all the columns from a specific table, only ask for the relevant columns given the question.
    You have access to tools for interacting with the database.
    Only use the below tools. Only use the information returned by the below tools to construct your final answer.
    You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

    DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

    To start you should ALWAYS look at the tables in the database to see what you can query.
    Do NOT skip this step.
    Then you should query the schema of the most relevant tables.
    """

from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content=message_template)

### Bint tools to LLM

In [7]:
llm_with_tools = llm.bind_tools(tools)

In [8]:
# Node
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [9]:
# from typing_extensions import Annotated
# from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


# class QueryOutput(TypedDict):
#     """Generated SQL query."""
#     query: Annotated[str, ..., "Syntactically valid SQL query."]


# def write_query(state: State):
#     """Generate SQL query to fetch information."""
#     prompt = query_prompt_template.invoke(
#         {
#             "dialect": db.dialect,
#             "top_k": 10,
#             "table_info": db.get_table_info(),
#             "input": state["question"],
#         }
#     )
#     structured_llm = llm.with_structured_output(QueryOutput)
#     result = structured_llm.invoke(prompt)
    
#     return {"query": result["query"]}


# def execute_query(state: State):
#     """Execute SQL query."""
#     execute_query_tool = QuerySQLDatabaseTool(db=db)
#     return {"result": execute_query_tool.invoke(state["query"])}


# def generate_answer(state: State):
#     """Answer question using retrieved information as context."""
#     prompt = (
#         "Given the following user question, corresponding SQL query, "
#         "and SQL result, answer the user question.\n\n"
#         f'Question: {state["question"]}\n'
#         f'SQL Query: {state["query"]}\n'
#         f'SQL Result: {state["result"]}'
#     )
#     response = llm.invoke(prompt)
#     return {"answer": response.content}

### Build a LangGraph graph (router)

In [10]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

# graph_builder = StateGraph(State).add_sequence(
#     [write_query, execute_query, generate_answer]
# )
# graph_builder.add_edge(START, "write_query")
# graph = graph_builder.compile()


# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", END)
# builder.add_edge("tools", "tool_calling_llm")

# Compile graph
graph = builder.compile()

### Visual representation of the graph

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

### Invoke the graph by providing a question

In [ ]:
# for step in graph.stream(
#     {"question": "List the tables in the database"}, stream_mode="updates"
# ):
#     print(step)
messages = graph.invoke({"messages": "How many users are there?"})
for m in messages['messages']:
    m.pretty_print()